In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
from Bio import SeqIO
import inspect

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
FASTA_PATH = pathlib.Path(DATA_PATH).joinpath('uniprot-compressed_true_download_true_format_fasta_includeIsoform_tr-2023.02.17-22.00.02.49.fasta')
RESULT_PATH = DATA_PATH.joinpath('gene_symbol_protein_sequences.pkl')


In [2]:
variables = [
    'annotations',
    'dbxrefs',
    'description',
    'features',
    'id',
    'letter_annotations',
    'name',
    'seq'
 ]

records = []
for data in SeqIO.parse(FASTA_PATH, "fasta"):
    record={}
    for index, value in enumerate(re.findall(r'(?:(?<=^)[^|]+|(?<=\|)[^| ]+|(?<=\|)[^ ]+)|(?<=\ ).+?(?= \w\w\=|$)', data.description)):
        value = value.strip()
        if index == 0:
            record['db'] = value
        elif index == 1:
            record['UniqueIdentifier'] = value
        elif index == 2:
            record['EntryName'] = value
        elif index == 3:
            record['ProteinName'] = value
        else:
            k, v = re.split(r'\=', value)
            record[k] = v

    record['seq'] = str(data.seq)
    
    records.append(record)

In [3]:
df = pd.DataFrame(records)

df.to_pickle(RESULT_PATH)

In [4]:
pd.read_pickle(RESULT_PATH)

db UniqueIdentifier         EntryName  \
0      tr       A0A075F5C6  A0A075F5C6_MOUSE   
1      sp       A0A087WPF7       AUTS2_MOUSE   
2      tr       A0A087WPT2  A0A087WPT2_MOUSE   
3      tr       A0A087WPU4  A0A087WPU4_MOUSE   
4      tr       A0A087WRK1  A0A087WRK1_MOUSE   
5      tr       A0A087WRT4  A0A087WRT4_MOUSE   
6      tr       A0A087WSP5  A0A087WSP5_MOUSE   
7      sp       A0A088MLT8       IQIP1_MOUSE   
8      tr       A0A0A6YWL7  A0A0A6YWL7_MOUSE   
9      tr       A0A0A6YXJ2  A0A0A6YXJ2_MOUSE   
...    ..              ...               ...   
94776  tr           Z4YN41      Z4YN41_MOUSE   
94777  tr           Z4YN47      Z4YN47_MOUSE   
94778  tr           Z4YN62      Z4YN62_MOUSE   
94779  tr           Z4YN74      Z4YN74_MOUSE   
94780  tr           Z4YN77      Z4YN77_MOUSE   
94781  tr           Z4YN82      Z4YN82_MOUSE   
94782  tr           Z4YN86      Z4YN86_MOUSE   
94783  tr           Z4YN92      Z4YN92_MOUSE   
94784  tr           Z4YNA3      Z4YNA3_MOUSE   
94785  tr           Z4YNA9      Z4YNA9_MOUSE   

                                                                    ProteinName  \
0                                                   Heat shock factor protein 1   
1                                  Autism susceptibility gene 2 protein homolog   
2                                       Prostaglandin G/H synthase 2 (Fragment)   
3                                            FAT atypical cadherin 1 (Fragment)   
4                                                         Predicted gene, 20814   
5                                                       FAT atypical cadherin 1   
6                              Signal transducer and activator of transcription   
7                                    IQCJ-SCHIP1 readthrough transcript protein   
8                                              Tyrosine-protein kinase receptor   
9                               IQ motif-containing GTPase-activating protein 3   
...                                                                         ...   
94776                                            GAS2-like protein 1 (Fragment)   
94777                           SKI/DACH domain-containing protein 1 (Fragment)   
94778                                        Ataxin-7-like protein 3 (Fragment)   
94779                        Transmembrane protein 11, mitochondrial (Fragment)   
94780    SANT and BTB domain regulator of class switch recombination (Fragment)   
94781                                                    Syntaphilin (Fragment)   
94782  Mitochondrial import inner membrane translocase subunit Tim22 (Fragment)   
94783                                    A-kinase anchor protein 17B (Fragment)   
94784                                    MAU2 chromatid cohesion factor homolog   
94785                                                    cDNA sequence AB124611   

                 OS     OX           GN PE SV  \
0      Mus musculus  10090         Hsf1  1  1   
1      Mus musculus  10090        Auts2  1  2   
2      Mus musculus  10090        Ptgs2  4  1   
3      Mus musculus  10090         Fat1  1  1   
4      Mus musculus  10090      Gm20905  3  1   
5      Mus musculus  10090         Fat1  1  2   
6      Mus musculus  10090        Stat1  1  1   
7      Mus musculus  10090  Iqcj-Schip1  1  2   
8      Mus musculus  10090        Ntrk3  1  1   
9      Mus musculus  10090       Iqgap3  1  1   
...             ...    ...          ... .. ..   
94776  Mus musculus  10090       Gas2l1  1  1   
94777  Mus musculus  10090       Skida1  4  1   
94778  Mus musculus  10090      Atxn7l3  1  1   
94779  Mus musculus  10090       Tmem11  1  1   
94780  Mus musculus  10090        Sanbr  4  1   
94781  Mus musculus  10090         Snph  1  1   
94782  Mus musculus  10090       Timm22  1  1   
94783  Mus musculus  10090      Akap17b  1  1   
94784  Mus musculus  10090         Mau2  1  1   
94785  Mus musculus  10090     AB124611  1  1   

                                            